<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорт-необходимых-библиотек" data-toc-modified-id="Импорт-необходимых-библиотек-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорт необходимых библиотек</a></span></li><li><span><a href="#Объявление-функций" data-toc-modified-id="Объявление-функций-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Объявление функций</a></span><ul class="toc-item"><li><span><a href="#Проверка-загрузки" data-toc-modified-id="Проверка-загрузки-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Проверка загрузки</a></span></li><li><span><a href="#Очистка-текста" data-toc-modified-id="Очистка-текста-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Очистка текста</a></span></li><li><span><a href="#Детекция-языка-текста" data-toc-modified-id="Детекция-языка-текста-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Детекция языка текста</a></span></li><li><span><a href="#Извлечение-года-из-кода-isrc" data-toc-modified-id="Извлечение-года-из-кода-isrc-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Извлечение года из кода isrc</a></span></li><li><span><a href="#Оставляем-один-жанр" data-toc-modified-id="Оставляем-один-жанр-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Оставляем один жанр</a></span></li><li><span><a href="#Предобработка" data-toc-modified-id="Предобработка-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Предобработка</a></span></li></ul></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Предобработка данных</a></span></li></ul></div>

# Предварительная обработка данных

## Импорт необходимых библиотек

In [1]:
# импорты из стандартной библиотеки
import re
import warnings

# импорты сторонних библиотек
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
from langdetect import detect, LangDetectException

# настройки
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option("max_colwidth", 100)

# константы
PARSED_DATA_PATH = "../data/parsed_data/"
PROCESSED_DATA_PATH = "../data/processed_data/"

## Объявление функций

### Проверка загрузки

In [2]:
def display_dataset_info(dataset):
    """
    Функция проверки загрузки: вывод первых 5 строк, информации о датафрейме и наличии дубликатов.

    Параметры:

     dataset: исследуемый датасет.

    Возвращаемое значение:

     Первые 5 строк датафрейма, метод .info() и метод .describe(include="all")
    """
    print("-" * 35 + "SAMPLE" + "-" * 35)
    display(dataset.head())
    print()
    print("-" * 36 + "INFO" + "-" * 36)
    print()
    display(dataset.info())
    print("-" * 34 + "DESCRIBE" + "-" * 34)
    display(dataset.describe(include="all"))

### Очистка текста

In [3]:
def clean_text(text):
    """
    Функция для очистки текста.

    Параметры:

         text: исходный текст

    Возвращаемое значение:

         txt: обработанный текст
    """

    # тексте иногда встречаются 1 или две цифры в начале строки - их убираем
    if text[0].isdigit() and text[1].isdigit():
        text = text[2:]
    elif text[0].isdigit():
        text = text[1:]

    # комменты в квадратных скобках (например [Intro]) меняем на пробел
    txt = re.sub(r"\[[^\]]*\]", r" ", text)

    # комменты в круглых скобках со звездочками (например (*Whistling*)) меняем на пробел
    txt = re.sub(r"\([^)]*\)", r" ", txt)

    # шаблон поиска и замены между contributors ... lyrics с захватом самих слов и аналогичных
    txt = re.sub(
        r"(C|c)ontributor(s|)([^;]*)((L|l)yrics|(V|v)(S|s))", r" ", txt
    )

    # многоточие меняем на точку
    txt = re.sub(r"[\.]+", r".", txt)

    # оставшиеся знаки препинания кроме апострофа на пустую строку
    txt = re.sub(r"[^\w\d'\s]", r"", txt)

    # оставшиеся знаки препинания кроме апострофа на пустую строку
    txt = re.sub("\n", r" ", txt)

    # повторяющиеся пробелы меняем на один пробел
    txt = re.sub(r"([ ])\1", r" ", txt)

    # обрезка пробелов слева
    txt = txt.lstrip()

    # обрезка пробелов справа
    txt = txt.rstrip()

    # текст в нижний регистр
    txt = txt.lower()

    return txt

### Детекция языка текста

In [4]:
def detect_language(text):
    """
    Функция детекции языка текста, применяется к столбцу с текстом песни.
    Для детекции используется модуль langdetect.

    Параметры:

         text: исходный текст

    Возвращаемое значение:

         - 'UNKNOWN', если язык не определен;
         - language, если язык определен;
    """
    try:
        language = detect(text).upper()
    except LangDetectException:
        language = "UNKNOWN"  # Язык по умолчанию для случаев, когда определение не удалось
    return language

### Извлечение года из кода isrc

In [5]:
def extract_year_from_isrc(row):
    """
    Функция извлечения года регистрации, применяется к столбцу с кодом isrc.

    Параметры:

         row: код isrc

    Возвращаемое значение:

         final_year: год в типе int.

    """
    # список для двухтысячных годов с 0-го по 23-й
    twenties = [i for i in range(0, 24)]

    # есть коды, в которых присутсвуют дефисы - дефисы убираем, стандарная длина кода 12 символов
    if len(row) > 12:
        row = row.replace("-", "")

    # в некоторых кодах год начинается с 5-го символа, в некоторых с 6-го. Извлекаем год в виде двух
    # символов, например '00' или "22"
    if row[5].isalpha():
        year = int(row[6:8])
    else:
        year = int(row[5:7])

    final_year = ""

    # так как год имеет два символа, то как факт можно принять, что с 00 по 23 это двухтысячные годы,
    # а остальные это 19 + символы года.
    # проверка, что год в двухтысячных
    if year in twenties:
        year = str(year)

        # если год от 0 до 9, то конкатенируем '200' и цифру года, переводив в тип int
        if len(year) == 1:
            final_year = "200" + year
            return int(final_year)

        # если год от 10, то конкатенируем '20' и цифру года, переводив в тип int
        else:
            final_year = "20" + year
            return int(final_year)

    # если год от 24 до 99, то конкатенируем '19' и цифру года, переводив в тип int
    else:
        year = str(year)
        final_year = "19" + year
        return int(final_year)

### Оставляем один жанр

In [6]:
def leave_one_genre(row):
    """leave_one_genre()

    Аргументы:
        data (pd.Series): Получает на вход pd.Series с листами жанров

    Возвращает:
        pd.Series: Возвращает pd.Series с но вместо list теперь простая строка с одним жанром
    """

    return row.split(", ")[0]


### Предобработка

In [7]:
def preprocess_dataset(data=None):
    """
    Функция обработки датасета.

    Параметры:

         data: исходный датасет

    Возвращаемое значение:

         data: обработанный датасет
    """
    # Удаляем дубликаты по isrc
    data = data.drop_duplicates(subset=["isrc"])
    print(f"Дубликаты записей с одинаковым кодом isrc удалены.")

    # Удаляем ненужные столбцы
    data = data.drop(["original_track_id", "track_id", "language"], axis=1)

    # Удаляем пропуски
    print(
        f"Удалены не нужные в дальнейшем столбцы: 'original_track_id', 'track_id', 'language'"
    )

    # Удаление пропусков
    data = data.dropna()
    print(f"Пропуски удалены.")

    # Очистка текста
    tqdm.pandas(desc="Очистка текста.")
    data["text"] = data["text"].progress_apply(clean_text)

    # Детекция языка по тексту песни
    tqdm.pandas(desc="Определение языка.")
    data["language_text"] = data["text"].progress_apply(detect_language)

    # Вытаскиваем год регистрации трека из isrc
    tqdm.pandas(desc="Вытаскиваем год регистрации трека из isrc")
    data["year"] = data["isrc"].progress_apply(extract_year_from_isrc)

    # Оставляем только один жанр в столбце genres
    tqdm.pandas(desc="Оставляем один жарн в genres")
    data["genres"] = data["genres"].progress_apply(leave_one_genre)

    # Перекодировка таргета в 0 и 1
    data["track_remake_type"] = data["track_remake_type"].map(
        {"ORIGINAL": 1, "COVER": 0}
    )

    # Приведение duration к типу int
    data["duration"] = data["duration"].astype("int")

    # порядок колонок в удобном виде
    cols = [
        "isrc",
        "title",
        "genres",
        "year",
        "language_text",
        "dttm",
        "duration",
        "text",
        "track_remake_type",
    ]
    data = data[cols]

    # категориальные столбцы в тип категорию
    cat_features = ["isrc", "title", "genres", "year", "language_text"]
    data[cat_features] = data[cat_features].astype("category")

    print("Обработка завершена")

    return data

## Предобработка данных

**Загрузка спаршенных данных**

In [8]:
parsed_data = pd.read_pickle(
    PARSED_DATA_PATH + "parsed_data", compression="zip"
)

**Вызов функции `preprocess_dataset` для обработки данных**

In [9]:
processed_data = preprocess_dataset(data=parsed_data)


Дубликаты записей с одинаковым кодом isrc удалены.
Удалены не нужные в дальнейшем столбцы: 'original_track_id', 'track_id', 'language'
Пропуски удалены.


Оставляем один жарн в genres: 100%|██████████████████████████████████████████| 71115/71115 [00:00<00:00, 677302.51it/s]


Обработка завершена


**Сохранение обработанного датасета**

In [10]:
processed_data.to_pickle(
    PROCESSED_DATA_PATH + "processed_data", compression="zip"
)